In [1]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

подгружаем данные

In [2]:
data = pd.read_csv("modified_train.csv")
y = data['price_range'].map({0: 0, 1: 0, 2: 1, 3: 1})
x = data.drop('price_range',axis=1)

пишем функцию со всеми нужными метриками для оценки качества модели

In [3]:
from sklearn.linear_model import LogisticRegression as SLLogisticRegression
from sklearn.neighbors import KNeighborsClassifier as SLKNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier as SLDecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier as SLRandomForestClassifier

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import precision_score, recall_score, accuracy_score

def GetMetrics(model, x=x, y=y, splits=5, average='macro'):
    kFold = StratifiedKFold(n_splits=splits, random_state=123, shuffle=True)
    precision = np.zeros(splits)
    recall = np.zeros(splits)
    testAccuracy = np.zeros(splits)
    trainAccuracy = np.zeros(splits)
    execTime = np.zeros(splits)

    for i, (iTrain, iCheck) in enumerate(kFold.split(x, y)):
        xTrain = x.loc[iTrain].to_numpy()
        yTrain = y.loc[iTrain].to_numpy()
        xCheck = x.loc[iCheck].to_numpy()
        yCheck = y.loc[iCheck].to_numpy()

        execTime[i] = -time.time()

        model.fit(xTrain, yTrain)

        yCheckPredicted = model.predict(xCheck)
        yTrainPredicted = model.predict(xTrain)

        execTime[i] += time.time()

        precision[i] = precision_score(yCheck, yCheckPredicted, average=average)
        recall[i] = recall_score(yCheck, yCheckPredicted, average=average)
        trainAccuracy[i] = accuracy_score(yTrain, yTrainPredicted)
        testAccuracy[i] = accuracy_score(yCheck, yCheckPredicted)

    return {'precision': precision.mean(), 'recall': recall.mean(), 'train accuracy': trainAccuracy.mean(), 'test accuracy': testAccuracy.mean(), 'execution time': execTime.mean()}

In [4]:
import time

modelNames = ('LogisticRegression', 'DecisionTreeClassifier', 'RandomForestClassifier')
for i, modelName in zip(range(1, len(modelNames) + 1), modelNames):
    exec(f'from models import {modelName} as My{modelName}')

    exec(f'myModel = My{modelName}()')
    exec(f'slModel = SL{modelName}()')
    myMetrics = GetMetrics(myModel)
    slMetrics = GetMetrics(slModel)
    
    print(f'{i}. {modelName}')
    print('Метрики алгоритма в моем исполнении:')
    for keys,values in myMetrics.items():
        print(f'{keys}: {values}')
    print()
    
    print('Метрики алгоритма из sklearn:')
    for keys,values in slMetrics.items():
        print(f'{keys}: {values}')
    print()
    
    accuracyRatio = slMetrics['test accuracy'] / myMetrics['test accuracy']
    timeRatio = myMetrics['execution time'] / slMetrics['execution time']
    if accuracyRatio > 1:
        print(f'Алгоритм из sklearn работает в {accuracyRatio:.3f} раз точнее.')
        if timeRatio > 1:
            print(f'И в {timeRatio:.3f} раз быстрее.')
        elif timeRatio < 1:
            timeRatio = 1 / timeRatio
            print(f'Но в {timeRatio:.3f} раз медленнее.')
        else:
            print('И с той же скоростью, что и моя реализация.')
    elif accuracyRatio < 1:
        accuracyRatio = 1 / accuracyRatio
        print(f'Написапнный мной алгоритм работает точнее, чем алгоритм из sklearn в {accuracyRatio:.3f} раз.')
        if timeRatio > 1:
            print(f'Но в {timeRatio:.3f} раз медленнее.')
        elif timeRatio < 1:
            timeRatio = 1 / timeRatio
            print(f'И в {timeRatio:.3f} раз быстрее.')
        else:
            print('И с той же скоростью')
    else:
        print('Моя реализация работает с той же точностью, что и алгоритм из sklearn.')
        if timeRatio > 1:
            print(f'Но в {timeRatio:.3f} раз медленнее.')
        elif timeRatio < 1:
            timeRatio = 1 / timeRatio
            print(f'И в {timeRatio:.3f} раз быстрее.')
        else:
            print('И с той же скоростью')
    print()
    print()

1. LogisticRegression
Метрики алгоритма в моем исполнении:
precision: 0.8157760171156513
recall: 0.7804296482412061
train accuracy: 0.7790726817042606
test accuracy: 0.7804511278195488
execution time: 0.07797503471374512

Метрики алгоритма из sklearn:
precision: 0.8970583282942493
recall: 0.8952814070351758
train accuracy: 0.9011278195488723
test accuracy: 0.8952380952380953
execution time: 0.03424115180969238

Алгоритм из sklearn работает в 1.147 раз точнее.
И в 2.277 раз быстрее.


2. DecisionTreeClassifier
Метрики алгоритма в моем исполнении:
precision: 0.9131895177549639
recall: 0.911781407035176
train accuracy: 0.9157894736842106
test accuracy: 0.9117794486215539
execution time: 0.22711873054504395

Метрики алгоритма из sklearn:
precision: 0.9431557778040902
recall: 0.9428693467336684
train accuracy: 1.0
test accuracy: 0.9428571428571428
execution time: 0.0078048229217529295

Алгоритм из sklearn работает в 1.034 раз точнее.
И в 29.100 раз быстрее.


3. RandomForestClassifier
Метри